In [1]:
import json
import pandas as pd
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta

with open("./db_config.json", "r") as f:
    config = json.load(f)


In [ ]:
DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]

In [ ]:
def create_db_connection(host_name, user_name, user_password, db_name):
    """MySQL 데이터베이스 연결을 생성합니다."""
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database 접속에 성공했습니다.")
    except Error as e:
        print(f"에러발생: '{e}'")
    return connection

def close_db_connection(connection):
    """MySQL 데이터베이스 연결을 닫습니다."""
    if connection and connection.is_connected():
        connection.close()
        print("MySQL 접속이 종료되었습니다.")

def fetch_data_to_dataframe(connection, query):
    """주어진 SQL 쿼리를 실행하고 결과를 Pandas DataFrame으로 반환합니다."""
    cursor = connection.cursor(dictionary=True) # 결과를 딕셔너리 형태로 받기 위함
    df = pd.DataFrame()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
    except Error as e:
        print(f"에러발생: '{e}', 에러유발쿼리: {query}")
    finally:
        if connection.is_connected():
            cursor.close()
    return df

In [10]:
def load_stadium_data(connection):
    """stadium 테이블 데이터를 로드합니다."""
    query = "SELECT stadium_code, sports_type, stadium_name, region, address FROM stadium" # 
    stadium_df = fetch_data_to_dataframe(connection, query)
    print("Stadium data loaded successfully.")
    return stadium_df

def load_sports_game_data(connection):
    """sports_game 테이블 데이터를 로드하고 game_date를 datetime으로 변환합니다."""
    # 테이블 컬럼명은 설계안에 명시된 것을 사용합니다. 
    query = """
        SELECT 
            stadium_code, sports_type, game_date, day_of_week, 
            start_time, end_time, home_team_win, match_type 
        FROM sports_game
    """
    sports_game_df = fetch_data_to_dataframe(connection, query)
    if not sports_game_df.empty and 'game_date' in sports_game_df.columns:
        sports_game_df['game_date'] = pd.to_datetime(sports_game_df['game_date'])
    print("Sports game data loaded successfully.")
    return sports_game_df

def load_traffic_accident_data(connection):
    """traffic_accident 테이블 데이터를 로드하고 accident_date를 datetime으로 변환합니다."""
    # 테이블 컬럼명은 설계안에 명시된 것을 사용합니다. 
    query = """
        SELECT 
            region, accident_date, accident_count, death_count, injury_count
        FROM traffic_accident 
    """ 
    # 참고: 설계안의 traffic_accident 테이블에는 stadium_code도 있지만, 
    # 최종 데이터셋은 지역 기준이므로 여기서는 stadium_code를 직접 사용하지 않습니다.
    traffic_accident_df = fetch_data_to_dataframe(connection, query)
    if not traffic_accident_df.empty and 'accident_date' in traffic_accident_df.columns:
        traffic_accident_df['accident_date'] = pd.to_datetime(traffic_accident_df['accident_date'])
    print("Traffic accident data loaded successfully.")
    return traffic_accident_df

def load_weather_data(connection):
    """weather 테이블 데이터를 로드하고 weather_date를 datetime으로 변환합니다."""
    # 테이블 컬럼명은 설계안에 명시된 것을 사용합니다. 
    # 여기서는 일별 대표 날씨를 위해, 날짜와 지역별로 평균 기온과 총 강수량을 가져오는 것을 가정합니다.
    # 실제 쿼리는 DB 데이터 구조와 집계 방식에 따라 달라질 수 있습니다.
    # weather 테이블에는 시간별 데이터가 있을 수 있으므로, 일별로 집계해야 합니다.
    query = """
        SELECT 
            region, 
            DATE(weather_date) as weather_date, 
            AVG(temperature) as temperature, 
            SUM(precipitation) as precipitation,
            AVG(cloud_amount) as cloud_amount  -- weather_condition 생성에 사용될 수 있음
        FROM weather 
        GROUP BY region, TIME(weather_time)
    """
    # 위 쿼리는 weather_date가 DATETIME 또는 TIMESTAMP 타입일 경우를 가정합니다.
    # 만약 weather_date가 이미 DATE 타입이라면 DATE() 함수는 필요 없을 수 있습니다.
    weather_df = fetch_data_to_dataframe(connection, query)
    if not weather_df.empty and 'weather_date' in weather_df.columns:
        weather_df['weather_date'] = pd.to_datetime(weather_df['weather_date'])
    print("Weather data loaded successfully.")
    return weather_df

In [11]:
def create_daily_features_for_region_from_db(
    connection, target_region_code, start_date_str, end_date_str
):
    """
    특정 지역에 대해 날짜별로 경기, 사고, 날씨 정보를 DB에서 로드하여 통합 데이터프레임을 생성합니다.

    Args:
        connection: MySQL 데이터베이스 연결 객체
        target_region_code (str): 분석 대상 지역 코드 (예: '서울_송파구')
        start_date_str (str): 분석 시작 날짜 (YYYY-MM-DD)
        end_date_str (str): 분석 종료 날짜 (YYYY-MM-DD)

    Returns:
        pd.DataFrame: 통합된 특성 데이터프레임
    """
    
    # 1. DB에서 각 테이블 데이터 로드
    stadium_df = load_stadium_data(connection)
    sports_game_df = load_sports_game_data(connection)
    traffic_accident_df = load_traffic_accident_data(connection)
    weather_df = load_weather_data(connection)

    # 데이터 로드 실패 시 빈 데이터프레임 반환 (또는 오류 처리)
    if stadium_df.empty or sports_game_df.empty or traffic_accident_df.empty or weather_df.empty:
        print("Warning: One or more data tables could not be loaded or are empty. Returning empty DataFrame.")
        # 실제 운영시에는 어떤 데이터가 비어있는지 더 상세한 로깅이나 예외 처리가 필요합니다.
        # 여기서는 모든 테이블에 데이터가 있어야 분석이 가능하다고 가정합니다.
        # 만약 일부 데이터가 없어도 분석을 진행해야 한다면, 각 df.empty 체크 후 로직을 분기해야 합니다.
        # 예를 들어, 특정 날짜에 날씨 정보만 없거나, 사고 정보만 없을 수도 있습니다.
        # 이 경우 해당 필드를 NaN 등으로 채우고 진행할 수 있습니다.
        # 현재 로직은 아래에서 .empty 체크를 하므로, 여기서는 경고만 출력하고 진행합니다.
        pass


    all_features = []
    
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    region_stadium_codes = []
    if not stadium_df.empty:
        region_stadium_codes = stadium_df[stadium_df['region'] == target_region_code]['stadium_code'].tolist()

    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        
        # 경기 정보 집계
        game_count = 0
        sport_types_list = []
        has_playoff = 0
        
        if not sports_game_df.empty and region_stadium_codes: # sports_game_df가 비어있지 않고, 해당 지역에 경기장이 있을 때만
            games_on_date = sports_game_df[
                (sports_game_df['game_date'] == current_date) &
                (sports_game_df['stadium_code'].isin(region_stadium_codes))
            ]
            if not games_on_date.empty:
                game_count = len(games_on_date)
                sport_types_list = sorted(list(set(games_on_date['sports_type'].tolist())))
                if '플레이오프' in games_on_date['match_type'].unique(): # .unique() 사용 권장
                    has_playoff = 1
        sport_types_str = ",".join(sport_types_list) if sport_types_list else "없음"

        # 사고 건수 가져오기
        accident_count = 0 # 기본값 0으로 설정
        if not traffic_accident_df.empty:
            accident_data = traffic_accident_df[
                (traffic_accident_df['accident_date'] == current_date) &
                (traffic_accident_df['region'] == target_region_code)
            ]
            accident_count = accident_data['accident_count'].sum()

        # 날씨 정보 가져오기 및 weather_condition 생성
        temperature = None
        precipitation_sum = None # 일일 총 강수량
        weather_condition = "정보 없음" # 기본값
        
        if not weather_df.empty:
            weather_data_on_date = weather_df[
                (weather_df['weather_date'] == current_date) &
                (weather_df['region'] == target_region_code)
            ]
            if not weather_data_on_date.empty:
                # 일별 집계된 데이터를 사용하므로 첫 번째 행을 사용
                temperature = weather_data_on_date['temperature'].iloc[0]
                precipitation_sum = weather_data_on_date['precipitation'].iloc[0]
                
                # weather_condition 생성 로직 (예시)
                # (precipitation 컬럼은 SUM으로 집계된 값이라고 가정)
                if pd.notna(precipitation_sum): # precipitation_sum이 NaN이 아닐 때
                    if precipitation_sum > 0:
                        weather_condition = "비"
                    else: # 강수량 0일 때
                        # cloud_amount가 있다면 활용 가능
                        # cloud_amount = weather_data_on_date['cloud_amount'].iloc[0]
                        # if pd.notna(cloud_amount) and cloud_amount > 5: # 구름 많음 (기준은 임의)
                        # weather_condition = "흐림"
                        # else:
                        weather_condition = "맑음" 
                # 더 정교한 로직 (눈, 안개 등)은 추가 컬럼 및 조건 필요
        
        weekday_str = ['월', '화', '수', '목', '금', '토', '일'][current_date.weekday()]
        is_holiday = 1 if current_date.weekday() >= 5 else 0 # 주말을 임시 공휴일로 간주

        all_features.append({
            'date': date_str,
            'region_code': target_region_code,
            'accident_count': accident_count,
            'game_count': game_count,
            'sport_types': sport_types_str,
            'has_playoff': has_playoff,
            'temperature': temperature,
            'precipitation': precipitation_sum,
            'weather_condition': weather_condition,
            'is_holiday': is_holiday,
            'weekday': weekday_str
        })
        
        current_date += timedelta(days=1)
        
    return pd.DataFrame(all_features)

In [12]:
if __name__ == '__main__':
    # 1. 데이터베이스 연결 생성
    # 실제 운영시에는 아래 DB 접속 정보를 안전한 곳에서 불러오도록 수정하세요.
    db_connection = create_db_connection(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

    final_df = pd.DataFrame() # 최종 결과를 담을 데이터프레임 초기화

    if db_connection and db_connection.is_connected():
        # 2. 분석 대상 지역 및 날짜 범위 설정
        target_region = "서울_송파구" # 설계안 예시 지역 
        # 날짜 범위는 DB에 있는 데이터에 맞춰서 설정해야 합니다.
        # 예시로 2023년 데이터를 가정합니다.
        start_date = "2023-06-12" # 
        end_date = "2023-06-15"   # (예시 데이터 범위에 맞게 조정)

        # 3. 데이터 통합 함수 호출
        print(f"\nCreating dataset for {target_region} from {start_date} to {end_date} using database.")
        final_df = create_daily_features_for_region_from_db(
            db_connection, 
            target_region, 
            start_date, 
            end_date
        )

        # 4. 결과 출력
        if not final_df.empty:
            print(f"\n--- Final Dataset for {target_region} ({start_date} ~ {end_date}) ---")
            print(final_df)
        else:
            print(f"\n--- No data found or an error occurred for {target_region} ({start_date} ~ {end_date}) ---")

        # 5. 데이터베이스 연결 종료
        close_db_connection(db_connection)
    else:
        print("Database connection failed. Please check your connection details.")

MySQL Database connection successful

Creating dataset for 서울_송파구 from 2023-06-12 to 2023-06-15 using database.
Stadium data loaded successfully.
Sports game data loaded successfully.
Traffic accident data loaded successfully.
The error '1055 (42000): Expression #2 of SELECT list is not in GROUP BY clause and contains nonaggregated column 'aiclass2.weather.weather_date' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by' occurred while fetching data with query: 
        SELECT 
            region, 
            DATE(weather_date) as weather_date, 
            AVG(temperature) as temperature, 
            SUM(precipitation) as precipitation,
            AVG(cloud_amount) as cloud_amount  -- weather_condition 생성에 사용될 수 있음
        FROM weather 
        GROUP BY region, TIME(weather_time)
    
Weather data loaded successfully.

--- Final Dataset for 서울_송파구 (2023-06-12 ~ 2023-06-15) ---
         date region_code  accident_

In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               4 non-null      object
 1   region_code        4 non-null      object
 2   accident_count     4 non-null      int64 
 3   game_count         4 non-null      int64 
 4   sport_types        4 non-null      object
 5   has_playoff        4 non-null      int64 
 6   temperature        0 non-null      object
 7   precipitation      0 non-null      object
 8   weather_condition  4 non-null      object
 9   is_holiday         4 non-null      int64 
 10  weekday            4 non-null      object
dtypes: int64(4), object(7)
memory usage: 484.0+ bytes


In [14]:
final_df.describe()


,accident_count,game_count,has_playoff,is_holiday
count,4.0,4.0,4.0,4.0
mean,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0


In [15]:
final_df.head(10)

,date,region_code,accident_count,game_count,sport_types,has_playoff,temperature,precipitation,weather_condition,is_holiday,weekday
0,2023-06-12,서울_송파구,0,0,없음,0,None,None,정보 없음,0,월
1,2023-06-13,서울_송파구,0,0,없음,0,None,None,정보 없음,0,화
2,2023-06-14,서울_송파구,0,0,없음,0,None,None,정보 없음,0,수
3,2023-06-15,서울_송파구,0,0,없음,0,None,None,정보 없음,0,목


In [22]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta, time # date, time은 sports_game_df 샘플에서만 사용했었음

DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_PORT = config["DB_PORT"]

# SQLAlchemy 엔진 생성
# mysql-connector-python 사용 시:
engine_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
# pymysql 사용 시:
# engine_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(engine_url)
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
except Exception as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit()

# --- 데이터베이스에서 데이터 로드하는 함수 ---
def load_table_to_df(table_name, engine):
    """지정된 테이블에서 모든 데이터를 Pandas DataFrame으로 로드합니다."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        print(f"'{table_name}' 테이블 로드 완료. {len(df)} 행.")
        # 날짜/시간 컬럼 타입 변환 (필요시)
        if 'game_date' in df.columns:
            df['game_date'] = pd.to_datetime(df['game_date'])
        if 'accident_date' in df.columns:
            df['accident_date'] = pd.to_datetime(df['accident_date'])
        if 'weather_date' in df.columns:
            df['weather_date'] = pd.to_datetime(df['weather_date'])
        # start_time, end_time, weather_time은 문자열로 로드될 수 있으므로, 필요시 time 객체로 변환
        return df
    except Exception as e:
        print(f"'{table_name}' 테이블 로드 중 오류 발생: {e}")
        return pd.DataFrame() # 오류 발생 시 빈 DataFrame 반환

# --- 데이터 로드 ---
stadium_df = load_table_to_df('stadium', engine)
sports_game_df = load_table_to_df('sports_game', engine)
traffic_accident_df = load_table_to_df('traffic_accident', engine)
weather_df = load_table_to_df('weather', engine)

# stadium_df의 'region' 컬럼 형식이 "도시_구" 형태(예: "서울_송파구")인지 확인 및 필요시 변환
# 문서 예시의 region_code가 "서울_송파구" 형태이므로, stadium 테이블의 region도 이와 일치해야 함
# 만약 DB의 stadium.region 컬럼이 "서울 송파구"와 같이 공백으로 구분되어 있다면 변환 필요
if not stadium_df.empty and 'region' in stadium_df.columns:
    # 예시: "서울 송파구" -> "서울_송파구"
    # 실제 DB 데이터 형식에 맞게 조정 필요
    # stadium_df['region'] = stadium_df['region'].str.replace(' ', '_', n=1) # 첫번째 공백만 _로 변경
    
    # 좀 더 견고한 변환 (샘플 데이터 생성 로직에서 가져옴)
    def standardize_region(region_str):
        if pd.isna(region_str):
            return region_str
        parts = region_str.split(' ')
        if len(parts) == 2: # 예: "서울 송파구", "부산 연제구"
            return f"{parts[0]}_{parts[1]}"
        elif len(parts) == 1: # 예: "천안시" (구가 없는 경우)
            # 이 경우, region_code와 어떻게 매핑될지 정책 필요.
            # 여기서는 그대로 두거나, "도시_도시" 형태로 만들 수 있음 (예: "충남_천안시")
            # 문서의 '서울 송파구'와 같은 형태를 기준으로 한다면, 구 단위까지 있는 지역만 고려
            return region_str # 또는 특정 규칙 적용
        return region_str # 그 외의 경우

    # 만약 stadium 테이블의 region 컬럼이 "서울 송파구", "경기 안산시" 와 같은 형태라면 아래 변환 적용
    # stadium_df['region'] = stadium_df['region'].apply(lambda x: x.replace(' ', '_') if isinstance(x, str) and ' ' in x and not '_' in x else x)
    # stadium_df.loc[stadium_df['region'] == '서울 송파구', 'region'] = '서울_송파구' # 이런식으로 특정 값 보정
    # 이 부분은 실제 DB 데이터에 맞춰 조정해야 합니다.
    # 여기서는 이미 "서울_송파구"와 같은 형태로 저장되어 있다고 가정하고 넘어갑니다.
    print("stadium_df['region'] 컬럼 형식 확인 필요.")

MySQL 데이터베이스에 성공적으로 연결되었습니다.
'stadium' 테이블 로드 완료. 60 행.
'sports_game' 테이블 로드 완료. 3631 행.
'traffic_accident' 테이블 로드 완료. 34032 행.
'weather' 테이블 로드 완료. 454506 행.
stadium_df['region'] 컬럼 형식 확인 필요.


In [26]:
# 1. User Input (as per document: "데이타셋을 구성전에 지역정보(서울 송파구)를 입력 받음")
TARGET_REGION = "서울_송파구" # 예시: 분석 대상 지역
START_DATE = pd.to_datetime("2023-01-01") # 분석 시작일
END_DATE = pd.to_datetime("2024-12-31")   # 분석 종료일

# 2. Create Base Calendar DataFrame
date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
base_df = pd.DataFrame({'date': date_range})
base_df['region_code'] = TARGET_REGION

# 3. Process Stadium Data (to get stadium codes in the target region)
if not stadium_df.empty:
    stadiums_in_target_region = stadium_df[stadium_df['region'] == TARGET_REGION]
    stadium_codes_in_region = stadiums_in_target_region['stadium_code'].unique().tolist()
    print(f"\n{TARGET_REGION} 내 경기장 코드: {stadium_codes_in_region}")
else:
    stadium_codes_in_region = []
    print(f"\n{TARGET_REGION} 내 경기장 정보 없음 또는 stadium 테이블 로드 실패.")


# 4. Process Sports Game Data
if not sports_game_df.empty and stadium_codes_in_region:
    games_in_region_df = sports_game_df[sports_game_df['stadium_code'].isin(stadium_codes_in_region)]
    games_in_region_df = games_in_region_df.rename(columns={'game_date': 'date'})

    if not games_in_region_df.empty:
        game_summary_df = games_in_region_df.groupby('date').agg(
            game_count=('stadium_code', 'count'),
            sport_types_list=('sports_type', lambda x: list(set(x))),
            has_playoff_list=('match_type', lambda x: 1 if any('플레이오프' in str(mt).lower() for mt in x) else 0)
        ).reset_index()

        game_summary_df['sport_types'] = game_summary_df['sport_types_list'].apply(lambda x: ','.join(sorted(x)) if x else '없음')
        game_summary_df['has_playoff'] = game_summary_df['has_playoff_list']
        game_summary_df = game_summary_df[['date', 'game_count', 'sport_types', 'has_playoff']]
        
        base_df = pd.merge(base_df, game_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 경기 정보 없음.")
        base_df['game_count'] = 0
        base_df['sport_types'] = '없음'
        base_df['has_playoff'] = 0
else:
    print("sports_game_df 로드 실패 또는 대상 지역 내 경기장 없음.")
    base_df['game_count'] = 0
    base_df['sport_types'] = '없음'
    base_df['has_playoff'] = 0

base_df['game_count'] = base_df['game_count'].fillna(0).astype(int)
base_df['sport_types'] = base_df['sport_types'].fillna('없음')
base_df['has_playoff'] = base_df['has_playoff'].fillna(0).astype(int)


# 5. Process Traffic Accident Data
if not traffic_accident_df.empty:
    # traffic_accident_df의 'region' 컬럼도 TARGET_REGION과 동일한 형식이어야 함.
    # 예: "서울_송파구"
    accidents_in_region_df = traffic_accident_df[traffic_accident_df['region'] == TARGET_REGION]
    accidents_in_region_df = accidents_in_region_df.rename(columns={'accident_date': 'date'})

    if not accidents_in_region_df.empty:
        accident_summary_df = accidents_in_region_df.groupby('date').agg(
            accident_count_sum=('accident_count', 'sum')
        ).reset_index()
        accident_summary_df = accident_summary_df.rename(columns={'accident_count_sum': 'accident_count'})
        
        base_df = pd.merge(base_df, accident_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 교통사고 정보 없음.")
        base_df['accident_count'] = 0
else:
    print("traffic_accident_df 로드 실패.")
    base_df['accident_count'] = 0
    
base_df['accident_count'] = base_df['accident_count'].fillna(0).astype(int)


# 6. Process Weather Data
if not weather_df.empty:
    # weather_df의 'region' 컬럼도 TARGET_REGION과 동일한 형식이어야 함.
    weather_region_df = weather_df[weather_df['region'] == TARGET_REGION]
    weather_region_df = weather_region_df.rename(columns={'weather_date': 'date'})

    if not weather_region_df.empty:
        # 날씨 데이터는 하루에 여러 번 기록될 수 있으므로, 일별 집계 필요
        # (문서 예시에는 평균 기온, 총 강수량이므로 이를 따름)
        weather_summary_df = weather_region_df.groupby('date').agg(
            temperature=('temperature', 'mean'),
            precipitation=('precipitation', 'sum'),
            avg_cloud_amount=('cloud_amount', 'mean') # 대표 날씨 상태 추론용
        ).reset_index()

        def get_weather_condition(row):
            if pd.isna(row['precipitation']) and pd.isna(row['avg_cloud_amount']):
                return '정보없음' # 데이터가 없는 경우
            if row['precipitation'] > 0:
                return '비/눈'
            elif pd.notna(row['avg_cloud_amount']):
                if row['avg_cloud_amount'] >= 7: # (0-10 기준)
                    return '흐림'
                elif row['avg_cloud_amount'] >= 3:
                    return '구름조금' # 또는 '약간흐림' 등
                else:
                    return '맑음'
            return '정보없음' # 강수량 없고 구름 정보도 없는 경우
            
        weather_summary_df['weather_condition'] = weather_summary_df.apply(get_weather_condition, axis=1)
        weather_summary_df = weather_summary_df[['date', 'temperature', 'precipitation', 'weather_condition']]
        
        base_df = pd.merge(base_df, weather_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 날씨 정보 없음.")
        base_df['temperature'] = pd.NA
        base_df['precipitation'] = pd.NA
        base_df['weather_condition'] = '정보없음'
else:
    print("weather_df 로드 실패.")
    base_df['temperature'] = pd.NA
    base_df['precipitation'] = pd.NA
    base_df['weather_condition'] = '정보없음'


# 7. Add Date Features
weekday_map_kr = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
base_df['weekday'] = base_df['date'].dt.dayofweek.map(weekday_map_kr)

try:
    import holidays
    kr_holidays = holidays.KR(years=base_df['date'].dt.year.unique().tolist()) # unique().tolist() 추가
    base_df['is_holiday'] = base_df['date'].apply(lambda d: 1 if d in kr_holidays else 0).astype(int)
except ImportError:
    print("`holidays` 라이브러리가 설치되지 않았습니다. `pip install holidays`로 설치해주세요. 'is_holiday'는 0으로 처리됩니다.")
    base_df['is_holiday'] = 0
except Exception as e:
    print(f"공휴일 정보 처리 중 오류: {e}. 'is_holiday'는 0으로 처리됩니다.")
    base_df['is_holiday'] = 0


# 8. Finalize
final_df = base_df[[
    'date', 'region_code', 'accident_count', 'game_count', 'sport_types',
    'has_playoff', 'temperature', 'precipitation', 'weather_condition',
    'is_holiday', 'weekday'
]].copy() # SettingWithCopyWarning 방지를 위해 .copy() 사용

final_df['date'] = final_df['date'].dt.strftime('%Y-%m-%d')
final_df['temperature'] = pd.to_numeric(final_df['temperature'], errors='coerce').round(1)
final_df['precipitation'] = pd.to_numeric(final_df['precipitation'], errors='coerce').round(1)

print("\n--- Final Dataset ---")
print(final_df)

# --- 활용 가능 분석 예시 ---
print("\n--- 활용 가능 분석 예시 ---")
if not final_df.empty:
    avg_accidents_game_day = final_df[final_df['game_count'] > 0]['accident_count'].mean()
    avg_accidents_no_game_day = final_df[final_df['game_count'] == 0]['accident_count'].mean()
    #print(f"평균 사고 건수 (경기 있는 날): {avg_accidents_game_day:.2f if pd.notna(avg_accidents_game_day) else 'N/A'}")
    #print(f"평균 사고 건수 (경기 없는 날): {avg_accidents_no_game_day:.2f if pd.notna(avg_accidents_no_game_day) else 'N/A'}")

    playoff_days_df = final_df[(final_df['game_count'] > 0) & (final_df['has_playoff'] == 1)]
    if not playoff_days_df.empty:
        avg_accidents_playoff_day = playoff_days_df['accident_count'].mean()
        print(f"평균 사고 건수 (플레이오프 경기 있는 날): {avg_accidents_playoff_day:.2f if pd.notna(avg_accidents_playoff_day) else 'N/A'}")
    else:
        print("분석 기간/지역 내 플레이오프 경기 데이터 없음.")
else:
    print("생성된 최종 데이터셋이 비어있어 분석을 수행할 수 없습니다.")

# 데이터베이스 엔진 연결 종료 (스크립트가 끝나면 자동으로 닫히지만 명시적으로 호출 가능)
if 'engine' in locals() and engine:
    engine.dispose()
    print("\n데이터베이스 연결이 종료되었습니다.")


서울_송파구 내 경기장 코드: []
sports_game_df 로드 실패 또는 대상 지역 내 경기장 없음.
서울_송파구 내 해당 기간 교통사고 정보 없음.
서울_송파구 내 해당 기간 날씨 정보 없음.
`holidays` 라이브러리가 설치되지 않았습니다. `pip install holidays`로 설치해주세요. 'is_holiday'는 0으로 처리됩니다.

--- Final Dataset ---
           date region_code  accident_count  game_count sport_types  \
0    2023-01-01      서울_송파구               0           0          없음   
1    2023-01-02      서울_송파구               0           0          없음   
2    2023-01-03      서울_송파구               0           0          없음   
3    2023-01-04      서울_송파구               0           0          없음   
4    2023-01-05      서울_송파구               0           0          없음   
..          ...         ...             ...         ...         ...   
726  2024-12-27      서울_송파구               0           0          없음   
727  2024-12-28      서울_송파구               0           0          없음   
728  2024-12-29      서울_송파구               0           0          없음   
729  2024-12-30      서울_송파구               0           0          없음

In [ ]:
final_df.to_csv('2차데이터셋_송파구.csv', index=False)

In [27]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               731 non-null    object 
 1   region_code        731 non-null    object 
 2   accident_count     731 non-null    int32  
 3   game_count         731 non-null    int32  
 4   sport_types        731 non-null    object 
 5   has_playoff        731 non-null    int32  
 6   temperature        0 non-null      float64
 7   precipitation      0 non-null      float64
 8   weather_condition  731 non-null    object 
 9   is_holiday         731 non-null    int64  
 10  weekday            731 non-null    object 
dtypes: float64(2), int32(3), int64(1), object(5)
memory usage: 54.4+ KB


In [28]:
final_df.head(10)

,date,region_code,accident_count,game_count,sport_types,has_playoff,temperature,precipitation,weather_condition,is_holiday,weekday
0,2023-01-01,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,일
1,2023-01-02,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,월
2,2023-01-03,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,화
3,2023-01-04,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,수
4,2023-01-05,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,목
5,2023-01-06,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,금
6,2023-01-07,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,토
7,2023-01-08,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,일
8,2023-01-09,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,월
9,2023-01-10,서울_송파구,0,0,없음,0,NaN,NaN,정보없음,0,화
